In [1]:
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from config import Config
import torch

config = Config()
seed = config.seed
torch.manual_seed(seed)
device = config.device

In [10]:
dataset = Planetoid('./data', 'Cora')
data = dataset[0]

transform = T.Compose([T.NormalizeFeatures(), T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True, split_labels=True)])
train_data, val_data, test_data = transform(data)

torch.save(train_data, './data/Cora/split/train_data.pt')
torch.save(val_data, './data/Cora/split/val_data.pt')
torch.save(test_data, './data/Cora/split/test_data.pt')